<a href="https://colab.research.google.com/github/Thanneru2/Humor-Detection/blob/main/Lokesh_CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import pandas as pd
import glob
path = "/content/drive/MyDrive/Train data/train_data"

files = glob.glob(path + "/*.tsv")
df_list = []
for f in files:
    df = pd.read_csv(f, sep='\t', header=None)
    df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
print("Total rows after merge:", len(data))


Total rows after merge: 11321


In [22]:
print(df.head())

                    0                                                  1  2
0  675107617488936960  @midnight #RealisticResolutions Lose 10 pounds...  0
1  675058663942410240  #RealisticResolutions @midnight Not to make an...  0
2  675122601270882304  @midnight change my underwear everyday #Realis...  0
3  675084915667542018  Finish reading the boring book I started in HS...  0
4  675071890973712384  Find out what that green stuff in the sink is....  0


In [23]:
print("Number of columns:", data.shape[1])

Number of columns: 3


In [24]:
data.columns = ["tweet_id", "tweet_text", "label"]

In [25]:
print(data.columns)
print(data.head())

Index(['tweet_id', 'tweet_text', 'label'], dtype='object')
             tweet_id                                         tweet_text  \
0  748170855453073408         The Grundge 2  #BadMonsterMovies @midnight   
1  748204791172567041  @midnight #PointsMe Creature From The Black Ba...   
2  748184088364752896  The People Under the Overpass  #BadMonsterMovi...   
3  748187336333090816     The Mummy's Mommy #BadMonsterMovies .@midnight   
4  748183833443336193     Carnival of Values #BadMonsterMovies @midnight   

   label  
0      0  
1      0  
2      0  
3      0  
4      0  


In [26]:
data = data[data['label'] != 2]

In [27]:
import string
import re
def preprocess_text(text):
    if pd.isnull(text):
        return ""

    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = ' '.join(text.split())

    return text
data["cleaned_text"] = data["tweet_text"].apply(preprocess_text)
data[["tweet_text", "cleaned_text"]].head()


,tweet_text,cleaned_text
0,The Grundge 2 #BadMonsterMovies @midnight,the grundge badmonstermovies midnight
1,@midnight #PointsMe Creature From The Black Ba...,midnight pointsme creature from the black ball...
2,The People Under the Overpass #BadMonsterMovi...,the people under the overpass badmonstermovies...
3,The Mummy's Mommy #BadMonsterMovies .@midnight,the mummys mommy badmonstermovies midnight
4,Carnival of Values #BadMonsterMovies @midnight,carnival of values badmonstermovies midnight


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=300)

X = tfidf.fit_transform(data["cleaned_text"])
y = data["label"]
print("Shape of TF-IDF feature matrix:", X.shape)


Shape of TF-IDF feature matrix: (11220, 300)


**Train-Test Split**

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


Training samples: 8976
Testing samples: 2244


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


Logistic Regression Accuracy: 0.9197860962566845
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2064
           1       0.00      0.00      0.00       180

    accuracy                           0.92      2244
   macro avg       0.46      0.50      0.48      2244
weighted avg       0.85      0.92      0.88      2244



In [31]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.9131016042780749
              precision    recall  f1-score   support

           0       0.92      0.99      0.95      2064
           1       0.14      0.02      0.03       180

    accuracy                           0.91      2244
   macro avg       0.53      0.50      0.49      2244
weighted avg       0.86      0.91      0.88      2244



In [32]:

from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)

print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Naive Bayes Accuracy: 0.9197860962566845
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2064
           1       0.00      0.00      0.00       180

    accuracy                           0.92      2244
   macro avg       0.46      0.50      0.48      2244
weighted avg       0.85      0.92      0.88      2244



In [33]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report

print("Support Vector Machine (SVM) Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


Support Vector Machine (SVM) Accuracy: 0.9193404634581105
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2064
           1       0.00      0.00      0.00       180

    accuracy                           0.92      2244
   macro avg       0.46      0.50      0.48      2244
weighted avg       0.85      0.92      0.88      2244

